In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from numpy.random import multivariate_normal as mvnorm
from numpy.linalg import norm
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import invwishart, matrix_normal, multivariate_normal

# import matplotlib.pyplot as plt
# from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
#                                AutoMinorLocator)
# plt.rcParams['figure.facecolor'] = 'white'
# plt.rcParams['axes.facecolor'] = 'white'
# %matplotlib inline

import pdb, time

seed = 0
np.random.seed(seed)

In [2]:
# Fama French 5 regression to 30 industry portfolio
params = pd.read_pickle("/Users/sikai/Dropbox/Research/post_sum/code/dlm_code/ff5_30_standard_parameters.pkl")
n_draws = 4000

C = np.array(params['C'])
m = np.array(params['m'])
nu = np.array(params['nu'])
S = np.array(params['S'])
X = np.array(params['X'])
p = m.shape[2]

err_mean = np.zeros(p)

Y = []

for t in range(nu.shape[0]):
    W_t = invwishart.rvs(p + nu[t] - 1, nu[t] * S[t], n_draws)
    beta_t = np.array([matrix_normal.rvs(m[t].T, W_t[i], C[t]) for i in range(n_draws)])
    err_t = np.array([multivariate_normal.rvs(err_mean, W_t[i]) for i in range(n_draws)])
    Y_t = np.einsum('ijk, k -> ij', beta_t, X[t]) + err_t
    Y.append(Y_t)

Y = np.stack(Y)

import pickle
with open("/Users/sikai/Downloads/ff5_30_standard_4000_draws.pkl", 'wb') as pickle_file:
    pickle.dump(Y, pickle_file)

In [ ]:
def score(x, y):
    return np.trace(x @ y) - np.log(np.linalg.det(x))

In [ ]:
x = np.array([[1, 0.1], [0.1, 2]])
y_star = np.linalg.inv(x)
y_sparse= np.diag(np.diag(y_star))
print(y_sparse)
print(score(y_sparse, x))

def _f(b):
    return (score_sparse - x[0][0] * y_sparse[0][0] - x[1][1] * y_sparse[1][1] - 2 * x[0][1] * b[0] + np.log(np.linalg.det(y_sparse) - b[0] ** 2)) ** 2

b = minimize(_f, [0.5], method="Powell")
y_dense = y_star
y_dense[0][1] = y_dense[1][0] = b.x[0]
print(y_dense)
print(score(y_dense, x))

In [ ]:
x = np.array([[1, 0.5], [0.5, 1]])
y_old = np.linalg.inv(x)
y = y_old + 1e-2
print(score(y_old, x))
print(score(y, x) + np.trace((x - np.linalg.inv(y)) @ (y_old - y)))

In [ ]:
A = np.eye(3)
v = np.array([1.5, 1, 0.5]).reshape(3, 1)
nabla = v @ v.transpose()

In [ ]:
(
    np.linalg.inv(nabla.reshape(-1, 1) @ nabla.reshape(1, -1) 
               + np.eye(nabla.flatten().shape[0])
    ) @ A.flatten()
).reshape(3, 3)

In [ ]:
A - (np.sum(nabla * A) / (1 + np.sum(nabla * nabla))) * nabla